In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
from tensorflow.keras.applications.resnet50 import preprocess_input

# Your existing code for data preparation
data_folder = './data'
plant_ids = []
plant_types = []
image_paths = []
image_numbers = []
widths = []
heights = []
aspect_ratios = []
image_data = []

for plant_type in os.listdir(data_folder):
    plant_type_folder = os.path.join(data_folder, plant_type)    
    if os.path.isdir(plant_type_folder):
        for image_number, image_file in enumerate(os.listdir(plant_type_folder), start=1):
            image_path = os.path.join(plant_type_folder, image_file)            
            with Image.open(image_path) as img:
                width, height = img.size                
                aspect_ratio = width / height
                img_array = np.array(img.convert('RGB'))  # Convert to RGB if grayscale
                
            plant_ids.append(image_number)
            plant_types.append(plant_type)
            image_paths.append(image_path)
            image_numbers.append(image_number)
            widths.append(width)
            heights.append(height)
            aspect_ratios.append(aspect_ratio)
            image_data.append(img_array)

# Create a DataFrame
data = {
    'plant_id': plant_ids,
    'plant_type': plant_types,
    'Image': image_paths,
    'ImageNumber': image_numbers,
    'Width': widths,
    'Height': heights,
    'AspectRatio': aspect_ratios,
    'ImageData': image_data,
}

df = pd.DataFrame(data)

# Resize images to a consistent size
target_size = (224, 224)
df['ImageData'] = df['ImageData'].apply(lambda x: array_to_img(x).resize(target_size))
df['ImageData'] = df['ImageData'].apply(lambda x: img_to_array(x))
df['ImageData'] = df['ImageData'].apply(lambda x: preprocess_input(x))

# Convert plant types to numerical labels
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['plant_type'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['ImageData'].tolist(), df['Label'].tolist(), test_size=0.2, random_state=42)

# Convert to NumPy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define the ResNet model
base_model = ResNet50(include_top=False, input_shape=(target_size[0], target_size[1], 3), pooling='avg')

# Freeze the layers in the pre-trained ResNet50 model
base_model.trainable = False

# Fine-tune only the last few layers
fine_tune_at = 100
for layer in base_model.layers[-fine_tune_at:]:
    layer.trainable = True

# Add new classification layers
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


KeyboardInterrupt: 